### 🧱 Building a GPT

📘 This notebook contains the code for implementing a transformer neural network, trained on a small dataset, composed by Shakespeare literature. It is based on the awesome tutorial from Andrej Karpathy, available in his [Youtube Channel](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=227s)

In [1]:
# dataset available at https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
# let's look at the first 200 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
vocabulary = sorted(list(set(text))) # get all unique characters
vocabulary_size = len(vocabulary)
print(''.join(vocabulary))
print(vocabulary_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### 🔐 Tokenization
The tokenization process involves having an "encoder" and "decoder" that will transform an input (chunk of text) into a list of integers.

In this example, the vocabulary is built at word level, from the dataset. It is the total amount of distinct characters. The encoding is pretty simple: each character returns its index in a sorted list that represents the vocabulary.

- "Hello" will return a list with 5 elements.
- "Hello there" will return a list with 11 elements.

The elements inside the list can be integers between 0 and 64, because we only have 65 elements in the vocabulary.

There are other tokenization strategies and the most widely used are chunk-level (grup of characters) instead of letter or word level.

Google's tokenizer is SentencePiece, available [here](https://github.com/google/sentencepiece), and OpenAI's tokenizer is TikToken, available [here](https://github.com/openai/tiktoken)

There's a trade-off between the vocabulary size and the sequence length after encoding. In TikToken, you have 50257 tokens, which means that "Hello" will return a shorter encoder list when compared to our custom encoding process, which is at character level.

##### ❓ Open Questions
- How do you end up having inputs of the same length if the encoding returns lists of different size?
- Is the trade-off always applicable? Couldn't that depend on the input text you are trying to encode?

In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(vocabulary) }
itos = { i:ch for i,ch in enumerate(vocabulary) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
import tiktoken 

openai_encoder = tiktoken.get_encoding("gpt2")
print(f"Vocabulary size for OpenAI's GPT-2: {openai_encoder.n_vocab}")
print("Encoding with OpenAI's tiktoken returns shorter list")
print(openai_encoder.encode("hii there"))

Vocabulary size for OpenAI's GPT-2: 50257
Encoding with OpenAI's tiktoken returns shorter list
[71, 4178, 612]


In [6]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org

encoded_dataset = torch.tensor(encode(text), dtype=torch.long)
print(f'Our dataset is now completely encoded. Shape: {encoded_dataset.shape}, Type: {encoded_dataset.dtype}')
print(f"Let's look at the first 150 characters: {encoded_dataset[:150]}")

Our dataset is now completely encoded. Shape: torch.Size([1115394]), Type: torch.int64
Let's look at the first 150 characters: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13])


### 🧪 Train Test Split
We are now performing the usual train test split and something interesting about it is that train data is getting 90% of the dataset without shuffling. How do we deal with "shuffle" if text is supposed to be sorted in order for it to make sense? The data will be divided into "chunks" of a specific length and what the network will receive are randomly chosen "chunks" that represent the context for the model to predict what comes next. The chunks or context have a maximum length.

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(encoded_dataset)) # first 90% will be train, rest val
train_data = encoded_dataset[:n]
val_data = encoded_dataset[n:]

In [8]:
context_length = 8
train_data[:context_length+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

##### ❓ Why do we have (n+1) tokens if context length is n? The idea is to have 8 examples of "prediction". When "18" is presented, we want the model to predict "47", when "18, 47" is presented, we want the model to predict "56", and so on. With 9 characters, there are 8 prediction instances. Another question that can come up is why do we train the transformer with increasing length examples? The benefit of doing it this way is that the transformer can predict with different context length. If the model gets one token or multiple tokens, it can handle the situation at inference time because it was trained for it.

In [9]:
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
    context = x[:t+1] # all elements up to t (including it)
    target = y[t] # expected value at time t
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

print(f"The inputs and targets create {batch_size*block_size} independent examples that will be processed in parallel.")
print("These train examples being fed into the network are of variable length, as discussed before.")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
The inputs and targets create 32 independent examples that will be processed in parallel.
These train examples being fed into the network are of variable length, as discussed before.


### 🧠 Language Model
One of the simplest models to implement are N-gram models. The N stands for how many tokens are "looked at" in the sequence to predict the next token. That number is N-1 so, for example, in a Bigram Language Model, it looks to the previous token in order to predict.

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # we need to format the logits and targets properly for the loss function which excpects the Channels to be the second dimension
            logits = logits.view(B*T, C) # 32 x 65 dimension
            targets = targets.view(B*T) # 32 dimension
            loss = F.cross_entropy(logits, targets) # this is negative log likelihood loss

        # logits are scores, NOT probabilities
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocabulary_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print("We are expectinga loss of -ln(1/65) which is approximately 4.17")

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
We are expectinga loss of -ln(1/65) which is approximately 4.17


#### ❓ Logits will return a tensor of (B,T,C) dimensions. B corresponds to 4, T to 8, and C to 65. What do these dimensions represent?
- B: stands for batch size
- T: stands for time and is the same number as context length
- C: stands for channel and represents the vocabulary size

Logits will contain the scores for the next character in the sequence, in order to answer the question: Which character is more likely to be next?
Something important to notice in a bigram model is that the process is completely parallel because each token can infer what comes next by the fact that they know they are token X.

If I'm token 5, I can infer what comes next because it will be defined in the row corresponding to that index, when looking at the token_embedding_table.
If the input was only [5], then logits will be of shape 1x65 because it returns the fifth row in token_embedding_table. 
But, in our case, the entire input is of size BxT or 4x8 (32), as discussed before. This model is very simple which means that getting the actual probability for each item in the vocabulary is just looking up for that row in the table because there is no extra context I actually care about. This will change for models with greater context length.

In [12]:
print("Let's now generate some text, given a starting context. Note it is a stochastic model.")
batch = 2
time = 4
encoded_context = torch.full((batch, time), 24) # 24 is a valid index (between 0 and 65)
print(f"Encoded context: {encoded_context[0]} and decoded context is: {decode(encoded_context[0].tolist())}.")

max_new_tokens = 10
print(f"Time, of value {time}, controls the length of the context. Total sequence will have {time+max_new_tokens} tokens.")
generated_sequences = m.generate(encoded_context, max_new_tokens=max_new_tokens) # one for each element in batch
for sequence_index in range(batch):
    print(decode(generated_sequences[sequence_index].tolist()))


Let's now generate some text, given a starting context. Note it is a stochastic model.
Encoded context: tensor([24, 24, 24, 24]) and decoded context is: LLLL.
Time, of value 4, controls the length of the context. Total sequence will have 14 tokens.
LLLLbL?qP-QWkt
LLLLID&viYDEsx


#### ❓ Why does generate method get the entire context if this is a Bigram model?
It is a fair question to ask, given that prediction is based on the last element as the comment indicates in the function: we are focusing ONLY on the last time step. The idea is that this generate function intends to be general to be re-used later. When we create a model that uses a longer context.

#### ⚡ Training Process
Here detail the training process

In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # this should be smaller for larger models

In [14]:
batch_size = 32
for steps in range(15000): # increase number of steps for good results...

    # sample a batch of data
    input_batch, target_batch = get_batch('train')

    # evaluate the loss
    logits, loss = m(input_batch, target_batch)
    optimizer.zero_grad(set_to_none=True) # each step resets the gradients to compute them again
    loss.backward() # backpropagate the loss
    optimizer.step() # based on the gradients, take a step in the right direction

print(loss.item())

2.471229314804077


In [15]:
batch = 2
time = 1
encoded_context = torch.full((batch, time), 24) # 24 is a valid index (between 0 and 65)
print(f"Encoded context: {encoded_context[0]} and decoded context is: {decode(encoded_context[0].tolist())}.")

max_new_tokens = 300
generated_sequences = m.generate(encoded_context, max_new_tokens=max_new_tokens) # one for each element in batch
for sequence_index in range(batch):
    print(decode(generated_sequences[sequence_index].tolist()))


Encoded context: tensor([24]) and decoded context is: L.
LI eseear t by tit thodselegrt R:
Fo


r bunour ther w, wequghobatho s llops t t toras bd ily,
TEY ishwarod, se ttha's; I ppry memitth we ieeelo me,
Fait Cloog se tt afre ce tim wary stuklel d lofran VID g
OLI'tof areris nde imowlmandise wineatingiomanh y Mave,
NCHegr.
Trs y thaurymeresththonglast ff
LLorlout an
INSun IDUpod:
An ug t uly hme hondanok. orooniowor WAn; y tangghe-d e hixarr howinncotisharomatre he sw I ayoty he, o lyofug, we VINUCORIO,
Yofr Seret on melowe;
IO:
he'sghork.
SLow arehauginduld I w afoisor d
JAnfucuesond sayoullies ldiaritond sofay
Abothanses bledilin:
Whas?
BELUEELORLI


### 🎩 Mathematical Trick for Attention Mechanism

In this section, we want to get a way of the tokens "talking" to each other. This "talking" is limited to previous context. We do not have any information from the future because that is what we are trying to predict.

Let's imagine I am a token. One way to understand my past would be to average my channel with the cannels from the tokens in my history.
If the context is [14, 56, 19, 24, 32] and I am token "24", then I would average channel[14], channel[56], channel[19] and channel[24]. There are many downsides to this kind of interaction (for example, loss of spatial interaction) but let's assume we are using that for now.

Side not: bow is bag of words, an expression people use to indicate averaging a bunch of things together.

In [16]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
fake_logits = torch.randn(B,T,C)
fake_logits.shape

torch.Size([4, 8, 2])

In [17]:
# We want x[b,t] = mean_{i<=t} x[b,i]
fake_logits_averaged_with_previous_context = torch.zeros((B,T,C))
for batch_element in range(B): # each sample from the batch
    for time_step in range(T): # each time step
        fake_previous_logits = fake_logits[batch_element,:time_step+1] # (t,C)
        fake_logits_averaged_with_previous_context[batch_element, time_step] = torch.mean(fake_previous_logits, 0)

In [18]:
print(f"A sample from the batch:")
for time_step in range(T):
    print(f"Time {time_step}, logit: {fake_logits[0][time_step]}" \
        f", averaged: {fake_logits_averaged_with_previous_context[0][time_step]}")

A sample from the batch:
Time 0, logit: tensor([ 0.1808, -0.0700]), averaged: tensor([ 0.1808, -0.0700])
Time 1, logit: tensor([-0.3596, -0.9152]), averaged: tensor([-0.0894, -0.4926])
Time 2, logit: tensor([0.6258, 0.0255]), averaged: tensor([ 0.1490, -0.3199])
Time 3, logit: tensor([0.9545, 0.0643]), averaged: tensor([ 0.3504, -0.2238])
Time 4, logit: tensor([0.3612, 1.1679]), averaged: tensor([0.3525, 0.0545])
Time 5, logit: tensor([-1.3499, -0.5102]), averaged: tensor([ 0.0688, -0.0396])
Time 6, logit: tensor([ 0.2360, -0.2398]), averaged: tensor([ 0.0927, -0.0682])
Time 7, logit: tensor([-0.9211,  1.5433]), averaged: tensor([-0.0341,  0.1332])


The goal is to make this averaging more efficient. We can transform the operation into matrix multiplication by using triangular matrices.

In [19]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # this will give a lower triangular matrix
# we want to normalize each row because the result will be that each coefficient in the
# i-th row tells us how much each logit in the upper and current row contributes.
a = a / torch.sum(a, dim=1, keepdim=True) # dim means sum along the columns
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print(f"Original logits:\n{b}")
print(f"Weighted matrix:\n{a}")
print(f"Averaged logits:\n{c}")
print("These are for a batch of 1 element, context length 3 and vocabulary of 2 elements.")

Original logits:
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
Weighted matrix:
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
Averaged logits:
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
These are for a batch of 1 element, context length 3 and vocabulary of 2 elements.


In [20]:
lower_triangular = torch.tril(torch.ones(T, T))
weights = lower_triangular / torch.sum(lower_triangular, 1, keepdim=True)
print(weights.shape)

torch.Size([8, 8])


In [27]:
inputs, _ = get_batch('train') # 4 elements
logits, _ = m.forward(inputs, targets=None)
print("Original logits shape is {logits.shape}") # B, T, C

logits_averaged_with_previous_context = weights @ logits
# weights is (T, T) and logits is (B, T, C). What this matrix multiplication does is
# include a batch dimension in weights, making it (B, T, T) and then, for each batch element
# it computes the weighted average by multiplying the original weights (T, T) with the current (T, C)
# the result would be (B, T, C)
print(f"Averaged logits shape is {logits_averaged_with_previous_context.shape}") # B, T, C

Original logits shape is {logits.shape}
Averaged logits shape is torch.Size([32, 8, 65])


There's another way of computing the same operation, which is using the softmax operation. In this scenario, we get a sort of "lower triangular" matrix but, where the original one has zeros, this one has -inf, and where the original one has ones, this one has zeros. What softmax does is e^coefficient (e^0 is 1, e^-inf is 0) and then divide by the sum of elements in the row. Basically, this returns the exact same result.

❓The question is... why would you want to do it this way? What's different or more efficient compared to the previous one?

In [28]:
# another version is using softmax
weights_v2 = torch.zeros((T,T))
weights_v2 = weights_v2.masked_fill(lower_triangular == 0, float('-inf')) # the masked indicates we can't look into the future
weights_v2 = F.softmax(weights_v2, dim=-1)
logits_averaged_with_previous_context_v2 = weights_v2 @ logits
# this will return True if both elements are equal
print(torch.allclose(logits_averaged_with_previous_context, logits_averaged_with_previous_context_v2))

True


In our current version, there's no distinction in how interesting another token is for the token in question. This is what's called "affinity". So far, the "attention" is equal to all tokens in the previous context (including this one) and there's zero "attention" to the tokens in the future. The future can't communicate with the past. The conclusion here is that you can make weighted aggregations from the past by using matrices in the lower triangular format.